In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 331M/331M [00:02<00:00, 112MB/s]
100% 331M/331M [00:02<00:00, 124MB/s]


In [ ]:
!mkdir face

In [ ]:
import zipfile
zip = zipfile.ZipFile('utkface-new.zip', 'r')
zip.extractall('/content/face')
zip.close()

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
folder_path = '/content/face/utkface_aligned_cropped/UTKFace'
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [ ]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [ ]:
df.head()

,age,gender,img
0,20,1,20_1_1_20170112192922677.jpg.chip.jpg
1,19,1,19_1_3_20170119152816720.jpg.chip.jpg
2,55,1,55_1_0_20170117155022829.jpg.chip.jpg
3,27,0,27_0_0_20170117012808138.jpg.chip.jpg
4,40,1,40_1_1_20170113012546137.jpg.chip.jpg


In [ ]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Instead of combining labels into a single column, keep them separate
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],  # Specify separate columns
    target_size=(200, 200),
    batch_size=32,
    class_mode='raw'  # Use 'raw' for multiple numerical outputs
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],  # Specify separate columns
    target_size=(200, 200),
    batch_size=32,
    class_mode='raw'  # Use 'raw' for multiple numerical outputs
)

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [ ]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [ ]:
model.compile(
    optimizer='adam',
    loss={
        'age': 'mean_absolute_error',  # 'mae' is a shorthand; both are valid
        'gender': 'binary_crossentropy'
    },
    metrics={
        'age': ['mae'],  # Wrapping in a list allows flexibility for multiple metrics
        'gender': ['accuracy']
    },
    loss_weights={
        'age': 1.0,
        'gender': 99.0
    }
)

In [ ]:
# Training loop
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 228s 340ms/step - age_mae: 17.3083 - loss: 17.3083 - val_age_mae: 16.5772 - val_loss: 16.5772
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 210s 333ms/step - age_mae: 16.3055 - loss: 16.3055 - val_age_mae: 16.5801 - val_loss: 16.5801
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 209s 332ms/step - age_mae: 16.4995 - loss: 16.4995 - val_age_mae: 16.5767 - val_loss: 16.5767
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 208s 329ms/step - age_mae: 16.4278 - loss: 16.4278 - val_age_mae: 16.5751 - val_loss: 16.5751
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 331ms/step - age_mae: 16.3880 - loss: 16.3880 - val_age_mae: 16.5756 - val_loss: 16.5756
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 331ms/step - age_mae: 16.5152 - loss: 16.5152 - val_age_mae: 16.5752 - val_loss: 16.5752
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 206s 327ms/step - age_mae: 16.4073 - loss: 16.4073 - val_age_mae: 16.5756 - val_loss: 16.5756
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 207s 329ms/step - age_mae: 16.328